### install packages

In [ ]:
# pip install import-ipynb

In [59]:
import import_ipynb
import TESTCode_generate_csr
import numpy as np
from scipy.sparse import csr_matrix
import copy
import pandas as pd


### generate random sparse matrix

In [78]:
# generate a random sparse matrix as the full_sample_matrix
rnd_useit = TESTCode_generate_csr.RandomUserItemMatrix(num_items=1000, num_users=100000, density=0.05)
mat_y = rnd_useit.generateMatrix()          # this is the full sample matrix
y_df = rnd_useit.getMatinPandasFormat()

In [79]:
# generate a random sparse matrix as predictions_matrix based on full_sample_matrix

mat_pred = copy.deepcopy(mat_y)
# Get non-zero indices
rows, cols = mat_pred.nonzero()
non_zero_indices = list(zip(rows, cols))

# Randomly select 40% of nonzero indices
selected_indices = np.random.choice(len(non_zero_indices), size=int(0.4*mat_pred.nnz), replace=False)

# Set selected entries to zero
for idx in selected_indices:
    r, c = non_zero_indices[idx]
    mat_pred[r, c] = 0

# Eliminate explicit zeros to keep CSR structure clean
mat_pred.eliminate_zeros()


In [80]:
# generate a random sparse matrix as training_matrix based on full_sample_matrix (this matrix has to be less dense than prediction_matrix)

mat_train = copy.deepcopy(mat_y)
# Get non-zero indices
rows, cols = mat_train.nonzero()
non_zero_indices = list(zip(rows, cols))

# Randomly select 60% of nonzero indices
selected_indices = np.random.choice(len(non_zero_indices), size=int(0.6*mat_train.nnz), replace=False)

# Set selected entries to zero
for idx in selected_indices:
    r, c = non_zero_indices[idx]
    mat_train[r, c] = 0

# Eliminate explicit zeros to keep CSR structure clean
mat_train.eliminate_zeros()

In [81]:
mat_y.nnz, mat_pred.nnz, mat_train.nnz

(4877074, 2926245, 1950830)

In [82]:
mat_y.shape, mat_pred.shape

((100000, 1000), (100000, 1000))

In [ ]:
# mat_y_df = rnd_useit.getMatinPandasFormat()
# mat_pred_df = pd.DataFrame.sparse.from_spmatrix(mat_pred)
# mat_train_df = pd.DataFrame.sparse.from_spmatrix(mat_train)

### evaluation class and its methods

In [ ]:
# -*- coding: utf-8 -*-
"""

"""

class RecSysEvaluator:
    
    def __init__(self):
        '''
        initialize the RecSysEvaluator with whatever other vals you feel are appropriate
        Each class must have a dict to track RMSE, Precision@K, etc
        
        
        Returns
        -------
        None

        '''
        self.evaluation_scores = {} #create dictionary to track scores
        
        
        
    def calc_rmse(self, actual_matrix: csr_matrix, predictions_matrix: csr_matrix, training_matrix: csr_matrix) :
        '''
        

        Parameters
        ----------
        actual_matrix : csr_matrix (Sparse Scipy matrix)
            The ground truth matrix with all the actual ratings by the users we are analyzing. 
            Only use the random seed of the sample matrix, not the full massive ratings table.
            
        predictions_matrix : csr_matrix (Sparse Scipy matrix)
            The matrix that has been trained and is making predictions on ratings by users.
            This matrix can be the Simple Recommender, the Vanilla Matrix Factorization Model,
            or the Hybrid Matrix Factorization model. The RMSE calculations and processes should be the 
            same for all 3
            
        training_matrix: csr_matrix (Sparse Scipy matrix)
            The matrix that the model has been training on. Use this matrix to make sure you are excluding
            ratings that ALREADY existed prior to the recommender systems ratings. You can create a boolean mask
            or you can simply exclude any ratings already listed in this matrix so that only the new predicted
            ratings are being evaluated
            
            
        Edge Case to consider. What if the user made 0 new ratings in the test set? How do you calc RMSE?

        Returns
        -------
        Nothing. Update the dictionary value for the RMSE ex: self.evaluation_scores['RMSE'] = average_rmse_score 

        when doing rmse, you are porbably going to do full matrix - training matrix, 
        that in theory should give you testing matrix, 
        now that you have the testing samples you should do test rating value - predicted rating value, 
        to get how far away we are from the true data

        '''



        # calculate test_matrix
        test_matrix = actual_matrix - training_matrix

        # calculate the difference matrix between predictions and training
        pred_matrix = abs(predictions_matrix - training_matrix)

        # Compute difference
        diff = test_matrix - pred_matrix

        # Compute squared error only on non-zero entries
        squared_error = diff.multiply(diff)  

        # Sum of squared errors
        sum_squared_error = squared_error.sum()

        # Number of elements (total, not just non-zero)
        n_elements = diff.nnz

        # RMSE
        rmse = np.sqrt(sum_squared_error / n_elements)

        # add to RMSE dict
        self.evaluation_scores["RMSE"] = rmse

        
    
    
    def calc_mae(self, actual_matrix: csr_matrix, predictions_matrix: csr_matrix, training_matrix: csr_matrix): 
        '''
        
        Parameters
        ----------
        Calculate the MAE as well just so that we can switch between the two if we need to.
        Process will be exactly the same as the RMSE calculation
        
        actual_matrix : csr_matrix (Sparse Scipy matrix)
            The ground truth matrix with all the actual ratings by the users we are analyzing. 
            Only use the random seed of the sample matrix, not the full massive ratings table.
            
        predictions_matrix : csr_matrix (Sparse Scipy matrix)
            The matrix that has been trained and is making predictions on ratings by users.
            This matrix can be the Simple Recommender, the Vanilla Matrix Factorization Model,
            or the Hybrid Matrix Factorization model. The RMSE calculations and processes should be the 
            same for all 3
            
        training_matrix: csr_matrix (Sparse Scipy matrix)
            The matrix that the model has been training on. Use this matrix to make sure you are excluding
            ratings that ALREADY existed prior to the recommender systems ratings. You can create a boolean mask
            or you can simply exclude any ratings already listed in this matrix so that only the new predicted
            ratings are being evaluated
            
            
        Edge Case to consider. What if the user made 0 new ratings in the test set? How do you calc RMSE?
        Returns
        -------
        Nothing. Update the dictionary value for the MAE

        '''

        # calculate test_matrix
        test_matrix = actual_matrix - training_matrix

        # calculate the difference matrix between predictions and training
        pred_matrix = abs(predictions_matrix - training_matrix)

        
        # Compute difference
        diff = abs(test_matrix - pred_matrix)

        # Sum of absolute errors
        sum_abs_error = diff.sum()

        # Number of elements (total, not just non-zero)
        n_elements = diff.nnz

        # MAE
        mae = sum_abs_error / n_elements
        
        # add to MAE dict
        self.evaluation_scores["MAE"] = mae

        
    def calc_precision_recall_at_k(self, actual_matrix: csr_matrix, predictions_matrix: csr_matrix,training_matrix: csr_matrix, k, user_count, threshold = 3.5 ) :
        '''
        

        Parameters
        ----------
        actual_matrix : csr_matrix (Sparse Scipy matrix)
            The ground truth matrix with all the actual ratings by the users we are analyzing. 
            Only use the random seed of the sample matrix, not the full massive ratings table.
            
        predictions_matrix : csr_matrix (Sparse Scipy matrix)
            The matrix that has been trained and is making predictions on ratings by users.
            This matrix can be the Simple Recommender, the Vanilla Matrix Factorization Model,
            or the Hybrid Matrix Factorization model. The RMSE calculations and processes should be the 
            same for all 3
        training_matrix: csr_matrix (Sparse Scipy matrix)
            The matrix that the model has been training on. Use this matrix to make sure you are excluding
            ratings that ALREADY existed prior to the recommender systems ratings. You can create a boolean mask
            or you can simply exclude any ratings already listed in this matrix so that only the new predicted
            ratings are being evaluated
            
        k : integer
            number of movies that the trained recommender will output
        user_count: integer
            number of total users that this recommender is making a movie recommendation for

        Edge Case to consider. What if the user made 0 new ratings in the test set? How do you calc RMSE?
        Returns
        -------
        Nothing. Update the dictionary value for the Precision@K

        '''
        
        
        
        '''
        You will need to calculate the number of "relevant items". Relevant Items are the total number
        of movies that the user Actually rated in the test set. Exclude any of that users ratings in the 
        training set. If the recommender recommends 3 movies and only 1 out of those 3 recommended 
        movies was actually rated (is in the relevant items set) then Precision@K is 1/3=.333 
        
        Formula is Precision@K = (Number of relevant items in the top K recommendations)/ K
        
        You will also need to take the average Precision@K for all the users so that the final
        Precision@K score is the sum of all the Precision@K scores for ALL the users Divided by
        the total number of users
        
        '''
        
        n_users = user_count   # user_count can be removed as an input and test_matrix.shape[0] can be used to compute n_users
        precisions = []
        recalls = []

        # calculate test_matrix
        test_matrix = actual_matrix - training_matrix

        # calculate the difference matrix between predictions and training
        pred_matrix = abs(predictions_matrix - training_matrix)

        for user in range(n_users):
            # Get row slices for user
            test_row = test_matrix[user].toarray().ravel()
            pred_row = pred_matrix[user].toarray().ravel()

            # Relevant items in test (non-zero ratings)
            relevant_items = np.where(test_row > threshold)[0]

            if len(relevant_items) == 0:
                continue  # skip users with no relevant items

            # Top-K predicted items
            top_k_items = np.argsort(-pred_row)[:k]  # sort descending by prediction

            # Intersection of recommended and relevant
            recommended_relevant = np.intersect1d(top_k_items, relevant_items)

            # Precision@K
            precision = len(recommended_relevant) / k
            # Recall@K
            recall = len(recommended_relevant) / len(relevant_items)

            precisions.append(precision)
            recalls.append(recall)

        # Average across users
        avg_precision = np.mean(precisions) if precisions else 0.0
        avg_recall = np.mean(recalls) if recalls else 0.0

        self.evaluation_scores["Precision@K"] = avg_precision
        self.evaluation_scores["Recall@K"] = avg_recall


        
    def calc_recall_at_k(self, actual_matrix: csr_matrix, predictions_matrix: csr_matrix,training_matrix: csr_matrix, k, user_count ) :
        '''
        

        Parameters
        ----------
        actual_matrix : csr_matrix (Sparse Scipy matrix)
            The ground truth matrix with all the actual ratings by the users we are analyzing. 
            Only use the random seed of the sample matrix, not the full massive ratings table.
            
        predictions_matrix : csr_matrix (Sparse Scipy matrix)
            The matrix that has been trained and is making predictions on ratings by users.
            This matrix can be the Simple Recommender, the Vanilla Matrix Factorization Model,
            or the Hybrid Matrix Factorization model. The RMSE calculations and processes should be the 
            same for all 3
        training_matrix: csr_matrix (Sparse Scipy matrix)
            The matrix that the model has been training on. Use this matrix to make sure you are excluding
            ratings that ALREADY existed prior to the recommender systems ratings. You can create a boolean mask
            or you can simply exclude any ratings already listed in this matrix so that only the new predicted
            ratings are being evaluated
            
        k : integer
            number of movies that the trained recommender will output
            
        user_count: integer
            number of total users that this recommender is making a movie recommendation for
        
        Edge Case to consider. What if the user made 0 new ratings in the test set? How do you calc RMSE?
        Returns
        -------
        Nothing. Update the dictionary value for the Precision@K

        '''
        
        
        
        '''
        You will need to calculate the Total number of "relevant items". Relevant items are the total number of movies in the 
        test group that the user actually rated. If the recommender recommends 5 movies and only 1 of those 5 movies are 
        in that list of all new movies that the user rated in the test set (let's say the user rated 10 new movies in the test
        set), then the Recall would be 1/10 = .1
        
        formula for Recall@K = (Number of relevant items in the top K recommendations)/(Total number of relevant items)
        
        You will also need to take the average Recall@K for all the users so that the final
        Recall@K score is the sum of all the Recall@K scores for ALL the users Divided by
        the total number of users
        
        '''
        
        
    '''
    Stretch goal: calculate the NDCG@K. Lookup NDCG@K formula online to see how it is calculated
    '''    
    def calc_NDCG_at_k(self, actual_matrix: csr_matrix, predictions_matrix: csr_matrix,training_matrix: csr_matrix, k, user_count):
        '''
        

        Parameters
        ----------
        actual_matrix : csr_matrix (Sparse Scipy matrix)
            The ground truth matrix with all the actual ratings by the users we are analyzing. 
            Only use the random seed of the sample matrix, not the full massive ratings table.
            
        predictions_matrix : csr_matrix (Sparse Scipy matrix)
            The matrix that has been trained and is making predictions on ratings by users.
            This matrix can be the Simple Recommender, the Vanilla Matrix Factorization Model,
            or the Hybrid Matrix Factorization model. The RMSE calculations and processes should be the 
            same for all 3
            
        training_matrix: csr_matrix (Sparse Scipy matrix)
            The matrix that the model has been training on. Use this matrix to make sure you are excluding
            ratings that ALREADY existed prior to the recommender systems ratings. You can create a boolean mask
            or you can simply exclude any ratings already listed in this matrix so that only the new predicted
            ratings are being evaluated
            
        k : TYPE
            DESCRIPTION.
        user_count : TYPE
            DESCRIPTION.
        
        Edge Case to consider. What if the user made 0 new ratings in the test set? How do you calc RMSE?
        Returns
        -------
        None.

        '''
        
    

### TEST EVALUATION

In [83]:
eval = RecSysEvaluator()
eval.calc_rmse(mat_y, mat_pred, mat_train)
eval.calc_mae(mat_y, mat_pred, mat_train)
eval.calc_precision_recall_at_k(mat_y, mat_pred, mat_train, 10, 100000, threshold=3.5)

In [84]:
eval.evaluation_scores["RMSE"], eval.evaluation_scores["MAE"]

(np.float64(3.4239408488101444), np.float64(3.0735169509987803))

In [85]:
eval.evaluation_scores["Precision@K"], eval.evaluation_scores["Recalln@K"]

(np.float64(0.619888), np.float64(0.5288648883752431))